In [ ]:
# -----------------------------------------------------------------------------
# Gramatica de C++ para generar el codigo intermedio
# -----------------------------------------------------------------------------

import sys
sys.path.append('../..')

from sly import Lexer, Parser

class Tabla():
    contador = 0

    def insertVar(self, var, value):
        if var in tabla:
            raise NameError('Variable ' + "\"" + var + "\"" + ' declarada anteriormente')
        else:
            self.contador -= 4
            tabla[var] = [self.contador, value]
    
    def insertValue(self, var, value):
        if var in tabla:
            tabla[var][1] = value #Modificamos el campo valor de la variable. 
        else:
            raise NameError('Variable ' + "\"" + var + "\"" + 'no declarada') 
    
    def compVar(self, var):
        if var in tabla:
            return 't'
        else:
            return 'f'
               
    def cadena(self, var):
        if var in tabla:
            res = str(tabla[var]) + "(%ebp)"
        else: 
            res = "$" + str(var)
        return res
    
manejador = Tabla()
tabla = {} #Dictionary
        
class NodoSuma():
    def __init__(self, valor1, valor2):
        #Vamos a la tabla para la comprobacion de tipos.
        str1 = manejador.cadena(valor1)
        str2 = manejador.cadena(valor2)
        
        print("movl " + str1 + ", %eax")  
        print("addl " + str2 + ", %eax")
        print("movl %eax, " + str1) 

class NodoProducto():
    def __init__(self, valor1, valor2):
        str1 = manejador.cadena(valor1)
        str2 = manejador.cadena(valor2)
        print("movl " + str1 + ", %eax")  
        print("addl " + str2 + ", %eax")
        print("movl %eax, " + str1)

class NodoEqual():
    def __init__(self, valor1, valor2):
        print("operador ==")
        
class NodoNequal():
    def __init__(self, valor1, valor2):
        print("operador !=")

class NodoGreater():
    def __init__(self, valor1, valor2):
        print("operador >")
        
class CalcLexer(Lexer):
    tokens = {ID, TIPO, NUM, PLUS, MINUS, TIMES, DIVIDE, ASSIGN, LPAREN, RPAREN, EQUAL, NEQUAL ,GREATER,
              IF, ELSE, WHILE, LKEY, RKEY, COMA, END}
    ignore = ' \t'

    # Tokens
    ID = r'[a-zA-Z_][a-zA-Z0-9_]*'
    ID['int'] = TIPO
    ID['if'] = IF
    ID['else'] = ELSE
    ID['while'] = WHILE
    NUM = r'\d+'
    
    #Aritmetic
    PLUS = r'\+'
    MINUS = r'-'
    TIMES = r'\*'
    DIVIDE = r'/'
    GREATER = r'>'
    EQUAL = r'=='
    NEQUAL = r'!='
    ASSIGN = r'='
    LPAREN = r'\('
    RPAREN = r'\)'
    LKEY = r'{'
    RKEY = r'}'
    COMA = r','
    END = r';'
    

    # Ignored pattern
    ignore_newline = r'\n+'

    # Extra action for newlines
    def ignore_newline(self, t):
        self.lineno += t.value.count('\n')

    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1

class CalcParser(Parser):
    tokens = CalcLexer.tokens
    #manejador = Tabla()
    
    #precedence = (
    #    ('left', PLUS, MINUS),
    #    ('left', TIMES, DIVIDE),
    #    ('right', UMINUS)
    #    )

    def __init__(self):
        self.names = { }
    
    @_('f_linea statement')
    def entrada(self,p):
        pass
    
    @_('TIPO linea END f_linea')
    def f_linea(self, p):
        print('Instruccion correcta')
    
    @_('')
    def f_linea(self, p):
        pass
        
    @_('assig rest')
    def linea(self, p):
        pass
        #manejador.insertVar(p.ID) #Aquí almacenamos la variable.
    
    @_('ID rest')
    def linea(self, p):
        manejador.insertVar(p.ID, 0)
    
    @_('ID ASSIGN logic')
    def assig(self,p):
        manejador.insertVar(p.ID, p.logic)
        print(tabla[p.ID])
    
    @_(' ')
    def assig(self, p):
        pass
    
    @_('COMA linea')
    def rest(self, p):
        pass
    
    #@_('linea')
    #def rest(self, p):
    #    return p.linea;
    
    @_(' ')
    def rest(self,p):
        pass
    
    #@_('TIPO ID')# statement')
    #def ide(self, p):
    #    manejador.insertVar(p.ID) #Añadimos la variable a la tabla.
    #    try:
    #        return p.ID 
    #    except LookupError:
    #        print(f'Undefined name {p.ID!r}')
    #        return 0
    
    
    #@_('TIPO assignement')
    #def ide(self,p):
    #    manejador.insertVar(p.ID)
        
    #@_('statement')
    #def ide(self, p):
    #    return p.statement
    
    @_('ID ASSIGN logic END statement') 
    def statement(self, p): #Falta buscar el valor en el sistema
        manejador.insertValue(p.ID, p.logic)
        #tabla[p.ID][1] = p.logic
        print(tabla[p.ID])
        
    @_('logic')
    def statement(self,p):
        return p.logic
    @_(' ')
    def statement(self,p):
        pass
    
    @_('logic EQUAL logicpr')
    def logic(self, p):
        NodoEqual(p.logic, p.logicpr)
    
    @_('logic NEQUAL logicpr')
    def logic(self, p):
        NodoNequal(p.logic, p.logicpr)
    
    @_('logicpr')
    def logic(self,p):
        return p.logicpr
    
    @_('logicpr GREATER expr')
    def logicpr(self, p):
        NodoGreater(p.logicpr, p.expr)

    
    @_('expr')
    def logicpr(self, p):
        return p.expr
    
    @_('expr PLUS fact')
    def expr(self, p):
        a = NodoSuma(p.expr,p.fact)
        
        
    @_('expr MINUS fact')
    def expr(self, p):
        a = NodoResta(p.expr, p.fact)
        return a.result #p.expr0 - p.expr1

    @_('fact')
    def expr(self, p):
        return p.fact
    
    @_('fact TIMES basico')
    def fact(self, p):
        a = NodoProducto(p.fact, p.basico)
        #return a.result #p.expr0 * p.expr1

    @_('fact DIVIDE basico')
    def fact(self, p):
        a = fact(p.fact, p.basico)
        return a.result #p.expr0 / p.expr1
    
    @_('basico')
    def fact(self, p):
        return p.basico
    
    @_('MINUS basico')
    def basico(self, p):
        return -p.basico
    
    @_('ID') #Uso de la variable
    def basico(self, p): #Si el id no esta en la tabla, debe mostrar error.
        return p.ID

    @_('NUM') 
    def basico(self, p):
        #print("NUM = ", p.NUM)
        return int(p.NUM)
    
    @_('LPAREN expr RPAREN')
    def basico(self, p):
        return p.expr
    
if __name__ == '__main__':
    lexer = CalcLexer()
    parser = CalcParser()
    while True:
        try:
            text = input('calc > ')
        except EOFError:
            break
        if text:
            parser.parse(lexer.tokenize(text))